In [ ]:
import torch
from torch.nn import functional as F
import torch.nn as nn
B, T, C = 4, 8, 32
x = torch.randn(B, T, C)

head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False) # 这里可以理解成矩阵乘法，可以理解为Wq @ x得到query
value = nn.Linear(C, head_size, bias=False)
k = key(x) # (B, T, head_size)
q = query(x) # (B, T, head_size)
v = value(x) # (B, T, head_size)
wei = q @ k.transpose(-2, -1) # (B, T, T)

tril = torch.tril(torch.ones(T, T))
# wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float("-inf")) # 有点类似于mask的操作，当前输入只能看到当前和以前的输入
wei = F.softmax(wei, dim=-1) # (B, T, T)
out = wei @ v # (B, T, head_size)


# out = wei @ x # (B, T, T) @ (B, T, C)  --> (B, T, C)
out.shape

torch.Size([4, 8, 32])

In [3]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3621, 0.6379, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.4609, 0.4936, 0.0455, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5894, 0.0468, 0.0279, 0.3359, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0113, 0.1358, 0.5566, 0.1602, 0.1362, 0.0000, 0.0000, 0.0000],
        [0.3382, 0.2637, 0.0737, 0.0234, 0.2319, 0.0690, 0.0000, 0.0000],
        [0.2697, 0.2301, 0.0381, 0.0323, 0.3946, 0.0092, 0.0259, 0.0000],
        [0.0540, 0.0196, 0.0340, 0.1138, 0.0065, 0.6821, 0.0875, 0.0026]],
       grad_fn=<SelectBackward0>)